# REGRESSION

Consider the following database from he UCI databases repository https://archive.ics.uci.edu/dataset/29/computer+hardware


In [1]:
# install the  b package

!pip install ucimlrepo

In [75]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
computer_hardware = fetch_ucirepo(id=29) 
  
# data (as pandas dataframes) 
df = computer_hardware.data.features
df.dropna(inplace=True)

X = df.drop(['PRP', 'ERP'], axis=1)
y = df[['PRP', 'ERP']]
print(df.dtypes)

le = LabelEncoder()
X['VendorName'] = le.fit_transform(X['VendorName'])
X['ModelName'] = le.fit_transform(X['ModelName'])

# metadata 
#print(computer_hardware.metadata) 
  
# variable information 
#print(computer_hardware.variables) 

VendorName    object
ModelName     object
MYCT           int64
MMIN           int64
MMAX           int64
CACH           int64
CHMIN          int64
CHMAX          int64
PRP            int64
ERP            int64
dtype: object


The content of the Database are:

Features
1. vendor name: 30 
      (adviser, amdahl,apollo, basf, bti, burroughs, c.r.d, cambex, cdc, dec, 
       dg, formation, four-phase, gould, honeywell, hp, ibm, ipl, magnuson, 
       microdata, nas, ncr, nixdorf, perkin-elmer, prime, siemens, sperry, 
       sratus, wang)
2. Model Name: many unique symbols
3. MYCT: machine cycle time in nanoseconds (integer)
4. MMIN: minimum main memory in kilobytes (integer)
5. MMAX: maximum main memory in kilobytes (integer)
6. CACH: cache memory in kilobytes (integer)
7. CHMIN: minimum channels in units (integer)
8. CHMAX: maximum channels in units (integer)

Target 

9. PRP: published relative performance (integer)
10. ERP: estimated relative performance from the original article (integer)


Perform a regression using ANNs on the 1-8 features and compare your results with each of the target values. 


In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# worse horrible when x is not scaled
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# works worse when y is scaled
# scaler2 = StandardScaler()
# y_train = scaler2.fit_transform(y_train)
# y_test = scaler2.fit_transform(y_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32) 
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32) 
print(X_train)

class RegressionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(RegressionModel, self).__init__()
        self.hidden1 = nn.Linear(input_size, 64) 
        self.hidden2 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, output_size)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        output = torch.relu(self.output_layer(x))
        return output

input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = RegressionModel(input_size, output_size)

criterion = nn.L1Loss() # this is MAE
optimizer = optim.Adam(model.parameters(), lr=0.01)  

num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

with torch.no_grad():
    test_output = model(X_test)
#     test_output = torch.tensor(scaler2.inverse_transform(test_output.numpy()))
#     y_test_tensor = torch.tensor(scaler2.inverse_transform(y_test_tensor.numpy()))
    print(test_output[0:5])
    print(y_test[0:5])
    test_loss = criterion(test_output, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")
    

tensor([[-1.2403,  1.4201,  0.3916,  ..., -0.5159, -0.5472, -0.5159],
        [-1.3723,  0.4312, -0.3738,  ...,  3.1823,  0.6599,  2.3147],
        [ 0.4755,  1.2325, -0.5925,  ...,  0.0201,  0.3150, -0.0278],
        ...,
        [-0.0525,  0.8574, -0.2645,  ..., -0.4087, -0.5472,  0.1674],
        [ 1.2674, -0.6259, -0.1916,  ..., -0.1943, -0.5472, -0.1254],
        [ 0.2115, -1.2056,  3.2346,  ..., -0.6231, -0.5472, -0.7599]])
Epoch [100/1000], Loss: 13.9825
Epoch [200/1000], Loss: 11.6949
Epoch [300/1000], Loss: 9.6607
Epoch [400/1000], Loss: 8.2912
Epoch [500/1000], Loss: 7.1605
Epoch [600/1000], Loss: 6.2750
Epoch [700/1000], Loss: 5.3828
Epoch [800/1000], Loss: 5.0610
Epoch [900/1000], Loss: 4.4102
Epoch [1000/1000], Loss: 4.2909
tensor([[332.7304, 102.7241],
        [ 25.2158,  24.5428],
        [ 15.6947,  23.8827],
        [287.8379, 523.5867],
        [ 19.8373,  33.6080]])
tensor([[274., 102.],
        [ 30.,  25.],
        [ 22.,  25.],
        [915., 919.],
        [ 16.,